# Knowledge Graph (Org Chart)

### General Constants

In [19]:
POLITICAL_POSITIONS_FILE = "political_positions.csv"
MILITARY_POSITIONS_FILE = "military_position.csv"
POLITICAL_MILITARY_MERGED_FILE = "merged_political_military.csv"

### Dependencies

In [20]:
import sys
# https://rdflib.github.io/sparqlwrapper/
!{sys.executable} -m pip install sparqlwrapper
!{sys.executable} -m pip install pandas


from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import pandas as pd
from datetime import datetime


You should consider upgrading via the 'p:\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'p:\programs\python\python39\python.exe -m pip install --upgrade pip' command.


## Common Pandas Code

In [21]:
# Save the wikiData results to a csv file in the current directory
def save_wikiData_csv(processed_results, csv_file_name):
    df = pd.DataFrame.from_dict(processed_results)
    df.to_csv(csv_file_name, index = None)

## Extracting data from WikiData
SPARQL written in https://w.wiki/4QiZ and modified the python code from there.

## Constants for all Wiki Data SPARQL code

In [22]:
WIKIDATA_ENDPOINT_URL = "https://query.wikidata.org/sparql"

# Could either pass in JSON or CSV to setReturnFormat, 
# but CSV does not work for some reason, it returns an XML.
# Returns a dict.
def get_wikiData_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON) 
    return sparql.query().convert()

# Returns an array of dicts with only the necessary values
# Remove rows with results that do not have lavels.
def get_processed_wikiData_results(query):
    raw_wikiData_results = get_wikiData_results(WIKIDATA_ENDPOINT_URL, query)
    processed_results = []
    for result in raw_wikiData_results["results"]["bindings"]:
        processed_results_dict = {}
        all_labelled = True
        for key, value in result.items():
            if not checkIfLabelExists(value["value"]):
                all_labelled = False
                break
            processed_results_dict[key] = value["value"]
        if all_labelled: 
            processed_results.append(processed_results_dict)
    return processed_results

# Results with no labels have the pattern "Q123" where 123 is any number
# Return True if label exists, otherwise, False.
def checkIfLabelExists(value):
    split_Q = value.split("Q")
    # URLs are fine
    if "http" in value:
        return True
    if len(split_Q) == 2:
        try:
            integer_side = int(split_Q[1])
            print("Label does not exist for value", value)
            return False
        except:
            return True
    return True

#### Political Positions

In [23]:
print("Starting political cell")

# https://w.wiki/4QiZ
# Get all politicians who were in the party during this period.
# Leave military rank to other query.
# Communist Party of the Soviet Union wd:Q79854 
# Communist Party of China wd:Q17427 
# Workers' Party of Korea wd:Q49623

political_position_query = """
SELECT DISTINCT ?item ?itemLabel ?politicalPartyLabel ?positionHeldLabel ?positionStart ?positionEnd
WHERE {
      VALUES ?politicalParty {wd:Q79854 wd:Q17427 wd:Q49623}
      ?item wdt:P102 ?politicalParty .
      ?item p:P39 ?positionHeldStatement .
      ?positionHeldStatement ps:P39 ?positionHeld .
      ?positionHeldStatement pq:P580 ?positionStart .
      ?positionHeldStatement pq:P582 ?positionEnd .
      FILTER (YEAR(?positionStart) < 1960 ).
      FILTER (YEAR(?positionEnd) > 1945).
     SERVICE wikibase:label { bd:serviceParam wikibase:language 'en'.}
  }
"""

political_position_results_processed = get_processed_wikiData_results(political_position_query)
save_wikiData_csv(political_position_results_processed, POLITICAL_POSITIONS_FILE)
print("Saved political positions to {}".format(POLITICAL_POSITIONS_FILE))

Starting political cell
Label does not exist for value Q54875420
Label does not exist for value Q62566426
Label does not exist for value Q4376665
Label does not exist for value Q4163773
Label does not exist for value Q52361248
Label does not exist for value Q4140563
Label does not exist for value Q52361248
Label does not exist for value Q52361248
Label does not exist for value Q4376646
Label does not exist for value Q102397210
Label does not exist for value Q4415450
Label does not exist for value Q26241348
Label does not exist for value Q12079953
Label does not exist for value Q12080034
Label does not exist for value Q12160399
Label does not exist for value Q12157728
Label does not exist for value Q12170800
Label does not exist for value Q12111381
Label does not exist for value Q12094998
Label does not exist for value Q12113076
Label does not exist for value Q12094031
Label does not exist for value Q12140135
Label does not exist for value Q12122282
Label does not exist for value Q12144

#### Military Positions

In [24]:
print("Starting military cell")

# https://query.wikidata.org/#%23%20Testing%20get%20higher%20and%20lower%20rank%0A%23%20Get%20Military%20Ranks%20of%20the%20Peple%20within%20the%20Party%0A%23%20Not%20all%20politicians%20have%20a%20military%20rank%2C%20and%20if%20put%20together%2C%20the%20query%20times-out.%20%0A%23%20Communist%20Party%20of%20the%20Soviet%20Union%20wd%3AQ79854%20%0A%23%20Communist%20Party%20of%20China%20wd%3AQ17427%20%0A%23%20Workers%27%20Party%20of%20Korea%20wd%3AQ49623%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3FpoliticalPartyLabel%20%3FmilitaryRankLabel%20%3FmilitaryRankStart%20%3FismilitaryRankPIT%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%3FnextHighestMilitaryRankLabel%20%3FnextLowestMilitaryRankLabel%0AWHERE%20%7B%0A%20%20%20%20%20%20VALUES%20%3FpoliticalParty%20%7Bwd%3AQ79854%20wd%3AQ17427%20wd%3AQ49623%7D%0A%20%20%20%20%20%20%3Fitem%20wdt%3AP102%20%3FpoliticalParty%20.%0A%20%20%20%20%20%20%3Fitem%20p%3AP410%20%3FmilitaryRankStatement%20.%0A%20%20%20%20%20%20%3FmilitaryRankStatement%20ps%3AP410%20%3FmilitaryRank.%0A%20%20%20%20%20%20%0A%20%20%20%20%20%20%23%20Get%20start%20timings%0A%20%20%20%20%20%20OPTIONAL%7B%3FmilitaryRankStatement%20pq%3AP585%20%3FmilitaryRankPIT%20%7D.%0A%20%20%20%20%20%20OPTIONAL%7B%3FmilitaryRankStatement%20pq%3AP580%20%3FmilitaryRankStartNoStop%20%7D.%0A%20%20%20%20%20%20FILTER%20%28%20%21bound%28%3FmilitaryRankPIT%29%20%7C%7C%20YEAR%28%3FmilitaryRankPIT%29%20%3E%3D%201945%20%29.%0A%20%20%20%20%20%20FILTER%20%28%20%21bound%28%3FmilitaryRankPIT%29%20%7C%7C%20YEAR%28%3FmilitaryRankPIT%29%20%3C%3D%201960%20%29%20.%0A%20%20%20%20%20%20FILTER%20%28%20%21bound%28%3FmilitaryRankStartNoStop%29%20%7C%7C%20YEAR%28%3FmilitaryRankStartNoStop%29%20%3C%3D1960%20%29%20.%0A%20%20%20%20%20%20BIND%28IF%28BOUND%28%3FmilitaryRankPIT%29%2C%3FmilitaryRankPIT%2C%3FmilitaryRankStartNoStop%29%20AS%20%3FmilitaryRankStart%29.%0A%20%20%20%20%20%20FILTER%28BOUND%28%3FmilitaryRankStart%29%29%0A%20%20%20%20%20%20BIND%28IF%28BOUND%28%3FmilitaryRankPIT%29%2C%201%20%2C%200%29%20AS%20%3FismilitaryRankPIT%29.%0A%20%20%0A%20%20%20%20%20%20%23%20Get%20higher%20and%20lower%20ranks%0A%20%20%20%20%20%20OPTIONAL%7B%3FmilitaryRank%20wdt%3AP3730%20%3FnextHighestMilitaryRank%7D.%0A%20%20%20%20%20%20OPTIONAL%7B%3FmilitaryRank%20wdt%3AP3729%20%3FnextLowestMilitaryRank%7D.%0A%20%20%20%20%20%20%0A%20%20%20%20%20%20%23%20Uncomment%20this%20if%20there%20has%20to%20be%20a%20higher%20and%20lower%20rank%20available%0A%23%20%20%20%20%20%20%20FILTER%20%28BOUND%28%3FnextLowestMilitaryRank%29%20%7C%7C%20BOUND%28%3FnextHighestMilitaryRank%29%29%0A%0A%20%20%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%27en%27.%7D%0A%20%20%7D%0A# Get Military Ranks of the Peple within the Party
# Not all politicians have a military rank, and if put together, the query times-out. 
# Communist Party of the Soviet Union wd:Q79854 
# Communist Party of China wd:Q17427 
# Workers' Party of Korea wd:Q49623
military_position_query = """
SELECT DISTINCT ?item ?itemLabel ?politicalPartyLabel ?militaryRankLabel ?militaryRankStart ?ismilitaryRankPIT 
                    ?nextHighestMilitaryRankLabel ?nextLowestMilitaryRankLabel
WHERE {
      VALUES ?politicalParty {wd:Q79854 wd:Q17427 wd:Q49623}
      ?item wdt:P102 ?politicalParty .
      ?item p:P410 ?militaryRankStatement .
      ?militaryRankStatement ps:P410 ?militaryRank.
      
      # Get start timings
      OPTIONAL{?militaryRankStatement pq:P585 ?militaryRankPIT }.
      OPTIONAL{?militaryRankStatement pq:P580 ?militaryRankStartNoStop }.
      FILTER ( !bound(?militaryRankPIT) || YEAR(?militaryRankPIT) >= 1945 ).
      FILTER ( !bound(?militaryRankPIT) || YEAR(?militaryRankPIT) <= 1960 ) .
      FILTER ( !bound(?militaryRankStartNoStop) || YEAR(?militaryRankStartNoStop) <=1960 ) .
      BIND(IF(BOUND(?militaryRankPIT),?militaryRankPIT,?militaryRankStartNoStop) AS ?militaryRankStart).
      FILTER(BOUND(?militaryRankStart))
      BIND(IF(BOUND(?militaryRankPIT), 1 , 0) AS ?ismilitaryRankPIT).
  
      # Get higher and lower ranks
      OPTIONAL{?militaryRank wdt:P3730 ?nextHighestMilitaryRank}.
      OPTIONAL{?militaryRank wdt:P3729 ?nextLowestMilitaryRank}.
      
      # Uncomment this if there has to be a higher and lower rank available
      # FILTER (BOUND(?nextLowestMilitaryRank) || BOUND(?nextHighestMilitaryRank))

      SERVICE wikibase:label { bd:serviceParam wikibase:language 'en'.}
  }
"""

military_position_results_processed = get_processed_wikiData_results(military_position_query)
save_wikiData_csv(military_position_results_processed, MILITARY_POSITIONS_FILE)
print("Saved military positions to {}".format(MILITARY_POSITIONS_FILE))

Starting military cell
Label does not exist for value Q4135271
Label does not exist for value Q4135295
Label does not exist for value Q4135276
Label does not exist for value Q4135272
Label does not exist for value Q4135272
Label does not exist for value Q4135295
Label does not exist for value Q4135276
Label does not exist for value Q4135272
Label does not exist for value Q4135286
Label does not exist for value Q4135276
Label does not exist for value Q8289964
Label does not exist for value Q4054390
Label does not exist for value Q8313962
Label does not exist for value Q8314133
Label does not exist for value Q8294958
Label does not exist for value Q8289964
Label does not exist for value Q4054390
Label does not exist for value Q4135276
Label does not exist for value Q4054390
Label does not exist for value Q8313962
Label does not exist for value Q8314133
Label does not exist for value Q4135276
Label does not exist for value Q4135285
Label does not exist for value Q4096454
Label does not ex

## Combine Military and Political Values

Cannot seem to combine the queries together in SPARQL because of the timeout from WikiData (too much data to process).

In [25]:
# Common Headers 
ITEM = "item"
ITEM_LABEL = "itemLabel"
POLITICAL_PARTY_LABEL = "politicalPartyLabel"

# Military Headers
MILITARY_RANK_LABEL = "militaryRankLabel"
MILITARY_RANK_START = "militaryRankStart"
IS_MILITARY_POINT_IN_TIME = "ismilitaryRankPIT"
NEXT_HIGHEST_RANK = "nextHighestMilitaryRankLabel" 
NEXT_LOWEST_RANK = "nextLowestMilitaryRankLabel"

# Political Headers
POLITICAL_POSITION = "positionHeldLabel"
POLITICAL_POSITION_START = "positionStart"
POLITICAL_POSITION_END = "positionEnd"

In [26]:
MILITARY_END_TIME_KEY = "militaryEnd"
EARLIEST_YEAR = 1945
EARLIST_DATETIME = datetime(EARLIEST_YEAR, 1, 1)

# There are 2 types of military polition start dates
# Point in time and start date
# Start date implies that it can continue to be true after the date.
# Point in time implies that we cannot tell if the statement is true after the date.

# Returns a dict of the relevant procesed military rows
# Adds an end date to each of the start dates if any.
# Removes military positions that end before 1945 and have been overwritten
# eg start points 1943, 1944, 1950 remove the one from 1943 only
def process_military_rows(military_df, political_id):
    relevant_m_rows = military_df.loc[military_df[ITEM] == political_id]
    temp_processed_rows = []
    processed_rows = []
    
    for index, m_row in relevant_m_rows.iterrows():
        curr_dict = m_row.to_dict()
        curr_dict[MILITARY_END_TIME_KEY] = None
        temp_processed_rows.append(curr_dict)
    # sort by start date
    temp_processed_rows = sorted(temp_processed_rows, key=lambda m_dict: m_dict[MILITARY_RANK_START]) 

    # get the end time for each row
    for i in range(len(temp_processed_rows) -1):
        temp_processed_rows[i][MILITARY_END_TIME_KEY] = temp_processed_rows[i+1][MILITARY_RANK_START]
    
    # only return rows with end time greater than specified time
    for row in temp_processed_rows:
        if row[MILITARY_END_TIME_KEY] == None:
            processed_rows.append(row)
        else:
            if get_date_time_from_str(row[MILITARY_END_TIME_KEY]) > EARLIST_DATETIME:
                processed_rows.append(row)
    return processed_rows

def get_date_time_from_str(date_time_str):
    date_format = "%Y-%m-%dT%H:%M:%SZ"
    return datetime.strptime(date_time_str, date_format)

# Returns a list of dicts of the military row that corresponds to the political row. 
# A row corresponds if the two dates of appointment intersect.
# May have more than one military appointment that corresponds.
def get_corresponding_military_dict(political_id, political_row, processed_military_rows):
    relevant_rows = []
    for m_row in processed_military_rows:
        # Check if the political position start date is before the military rank start date
        if (get_date_time_from_str(political_row[POLITICAL_POSITION_START]) <=  
                get_date_time_from_str(m_row[MILITARY_RANK_START])):
            # Check if the political position end date is after the military rank start date
            if (get_date_time_from_str(political_row[POLITICAL_POSITION_END]) >=
                    get_date_time_from_str(m_row[MILITARY_RANK_START])):
                relevant_rows.append(m_row)
        else:
            # Check if the military position end date is after the political position start date
            if (m_row[MILITARY_END_TIME_KEY] == None or 
                get_date_time_from_str(m_row[MILITARY_END_TIME_KEY]) > 
                get_date_time_from_str(political_row[POLITICAL_POSITION_START])):
                relevant_rows.append(m_row)
    return relevant_rows


def get_political_dict_without_military(p_row):
    curr_dict = p_row.to_dict()
    curr_dict[MILITARY_RANK_LABEL] = None
    curr_dict[MILITARY_RANK_START] = None
    curr_dict[IS_MILITARY_POINT_IN_TIME] = None
    curr_dict[MILITARY_END_TIME_KEY] = None
    curr_dict[NEXT_HIGHEST_RANK] = None
    curr_dict[NEXT_LOWEST_RANK] = None
    return curr_dict

import copy
def get_military_dict_without_political(m_row):
    if type(m_row) != dict:
        curr_dict = m_row.to_dict()
    else:
        curr_dict = copy.deepcopy(m_row) 
    curr_dict[MILITARY_END_TIME_KEY] = curr_dict.get(MILITARY_END_TIME_KEY, None)
    curr_dict[POLITICAL_POSITION] = None
    curr_dict[POLITICAL_POSITION_START] = None
    curr_dict[POLITICAL_POSITION_END] = None
    return curr_dict

def get_merged_political_military_dict(p_row, m_dict):
    p_dict = p_row.to_dict()
    return {**p_dict, **m_dict}

In [27]:
print("Starting merge cell")

merged_output = []

# Read the data from the original csv files
political_df = pd.read_csv(POLITICAL_POSITIONS_FILE)
military_df = pd.read_csv(MILITARY_POSITIONS_FILE)

# Get a list of the personel ids for each group
politician_ids = political_df[ITEM].unique()
military_ids = military_df[ITEM].unique()

# For each name in political csv, check if there is a corresponding military row
for political_id in politician_ids:
    # Check if a corresponding military id exists
    relevant_p_rows = political_df.loc[political_df[ITEM] == political_id]    
    if political_id in military_ids:
        processed_military_rows = process_military_rows(military_df, political_id)
        # Keep track of military rows were used
        used_military_rows_ids = set()
        
        for index, p_row in relevant_p_rows.iterrows():
            military_dict_list = get_corresponding_military_dict(political_id, p_row, processed_military_rows)
            if len(military_dict_list) == 0:
                curr_dict = get_political_dict_without_military(p_row)
                merged_output.append(curr_dict)
            else:
                for m_dict in military_dict_list:
                    used_military_rows_ids.add(id(m_dict))
                    curr_dict = get_merged_political_military_dict(p_row, m_dict)
                    merged_output.append(curr_dict)
                    
        # Check if the remaining military rows for the given id should go in without a corresponding political row
        for m_dict in processed_military_rows:
            if id(m_dict) not in used_military_rows_ids:
                curr_dict = get_military_dict_without_political(m_dict)
                merged_output.append(curr_dict)        
        continue
    
    # No corresponding military personel
    for index, p_row in relevant_p_rows.iterrows():
        curr_dict = get_political_dict_without_military(p_row)
        merged_output.append(curr_dict)


# if no political names correspond, add military label directly to the csv
for military_id in military_ids:
    if military_id not in politician_ids:
        relevant_m_rows = military_df.loc[military_df[ITEM] == military_id]
        for index, m_row in relevant_m_rows.iterrows():
            curr_dict = get_military_dict_without_political(m_row)
            merged_output.append(curr_dict)

save_wikiData_csv(merged_output, POLITICAL_MILITARY_MERGED_FILE)
print("Saved merged political and military positions to {}".format(POLITICAL_MILITARY_MERGED_FILE))

Starting merge cell
Saved merged political and military positions to merged_political_military.csv


## Add hierachy

In [28]:
# TODO IF NEEDED